In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

In [20]:
import sys
import yaml
from pathlib import Path

# ..src/
# __file__ is not defined
__file__ = os.path.abspath('.')

sys.path.append(os.path.abspath(os.path.join(os.path.dirname(__file__), '.')))
from src.functions import compliance_classifier
from src.functions import translation_format

prompts = yaml.safe_load(Path("../src/prompts.yml").read_text())


In [12]:
system_prompt = prompts["prompts"]["extractor"]["system_prompt"]
user_prompt = prompts["prompts"]["extractor"]["user_prompt"]

In [13]:
# Question related to compliance and personal data protection law
question = "What provisions are made in the personal data protection law for the rights of data subjects regarding access to their personal data? And from Articel 1, and Paragraphs 4, 5 and 6. What can you say? What about paragraph 6 in Article 23"

In [14]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=OPENROUTER_API_KEY,
)

# First API call with reasoning
response = client.responses.create(
  model="openai/gpt-oss-20b:free",
  input=[
          # Define the system prompt
          {
            "role": "system",
            "content": system_prompt
          },
          {
            "role": "user",
            "content": user_prompt.replace("{{ document_text }}", question)
          }
        ],
  extra_body={"reasoning": {"enabled": True}},
  text = compliance_classifier
)



In [15]:
import json
# Extract the assistant message with reasoning_details

response_dict = json.loads(response.output_text)
response_dict

{'articles': [], 'sectors': []}

In [25]:
def arabic_to_english_translation(text: str) -> str:
    """
    Translate Arabic text to English.

    Args:
        text (str): The input Arabic text.

    Returns:
        str: The translated English text.
    """
    system_prompt = prompts["prompts"]["translate_ar_en"]["system_prompt"]
    user_prompt = prompts["prompts"]["translate_ar_en"]["user_prompt"]

    response = client.responses.create(
        model="openai/gpt-oss-20b:free",
        input=[
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": user_prompt.replace("{{ arabic_text }}", text)
                }
            ],
        extra_body={"reasoning": {"enabled": True}},
        text = translation_format
    )

    response_dict = json.loads(response.output_text)
    return response_dict




In [26]:
translation_format

{'format': {'type': 'json_object',
  'json_schema': {'name': 'translation_only',
   'strict': True,
   'schema': {'type': 'object',
    'properties': {'translation': {'type': 'string'}},
    'required': ['translation'],
    'additionalProperties': False}}}}

In [27]:
arabic_text = "ما الأحكام الواردة في قانون حماية البيانات الشخصية بشأن حقوق صاحب البيانات في الوصول إلى بياناته الشخصية؟ بالنظر إلى المادة 1 (الفقرات 4 و5 و6) ما الذي تستخلصه؟ وبالنسبة للفقرة 6 من المادة 23، ماذا تتضمن بالضبط؟"
translation = arabic_to_english_translation(arabic_text)
# print(json.dumps(translation, indent=2))
print(translation)

{'translation': "What provisions are included in the Personal Data Protection Law regarding the data subject's right to access their personal data? Considering Article\u202f1 (paragraphs\u202f4,\u202f5, and\u202f6), what can be inferred? And, with respect to paragraph\u202f6 of Article\u202f23, what does it specifically contain?"}


In [29]:
print(translation["translation"])

What provisions are included in the Personal Data Protection Law regarding the data subject's right to access their personal data? Considering Article 1 (paragraphs 4, 5, and 6), what can be inferred? And, with respect to paragraph 6 of Article 23, what does it specifically contain?


In [12]:
print(translation)

{"assistantWhat provisions are contained in the Personal Data Protection Law regarding the rights of the data subject to access their personal data? Referring to Article 1 (paragraphs 4, 5, and 6), what do you infer? And regarding paragraph 6 of Article 23, what does it contain exactly?}What provisions are contained in the Personal Data Protection Law regarding the rights of the data subject to access their personal data? Referring to Article 1 (paragraphs 4, 5, and 6), what do you infer? And regarding paragraph 6 of Article 23, what does it contain exactly?}We are done.What provisions are contained in the Personal Data Protection Law regarding the rights of the data subject to access their personal data? Referring to Article 1 (paragraphs 4, 5, and 6), what do you infer? And regarding paragraph 6 of Article 23, what does it contain exactly?}What provisions are contained in the Personal Data Protection Law regarding the rights of the data subject to access their personal data? Referrin